# Episim.jl

Example of use and application for disease modelling via complex network contagion models.

In [1]:
include("EpiSim.jl")

Main.EpiSim

In [2]:
using CSV
using Plots
using LightGraphs
using JLD2, FileIO

First, we will get some data (always useful). In this case Australian infection numbers from the John Hopkins github repository

In [5]:
cd("/Users/michael/work/GitHub/COVID-19/csse_covid_19_data/csse_covid_19_time_series")
file="time_series_covid19_confirmed_global.csv"
allstates=["Australian Capital Territory" "New South Wales"  "Northern Territory" "Queensland" "South Australia" "Tasmania" "Victoria" "Western Australia"]
z=[]
for state in allstates
    z=push!(z,EpiSim.getdata("Australia",state))
end

In [6]:
pops=[426709 8089526 245869 5095100 1751693 534281 6594804 2621680]
ddays=CSV.File(file)[1]
ddays=propertynames(ddays,4)[5:end]
ddays=String.(ddays)
ndays=length(ddays)

99

In [7]:
cd("/Users/michael/work/GitHub/epinets")

In [ ]:
We need to prepare our model parameters. In this case, p0 and p2 and r0 and r2 indicate values before 

In [ ]:
nsim=50 #50 simulations
predays=50 #50 days bfore changepoint
postdays=150 #50 days after

#initialise and get set to track the various tallies
ndays=predays+postdays
st=Array{UInt64,2}(undef,ndays,nsim)
ex=Array{UInt64,2}(undef,ndays,nsim)
fe=Array{UInt64,2}(undef,ndays,nsim)
rm=Array{UInt64,2}(undef,ndays,nsim)

net1=
net2=

p0=
p2=
q=
r0=
r2=

In [ ]:
for j in 1:nsim  #perform nsim simualtions
    
    net=net1     #active contact network is net1 to start
    #initilise the rate parameters
    r=r0
    p=p0

    #reinitialise the state vector
    state=Array{Int8,2}(undef,1,pop) #this is a bit wasteful, there must be a better categorical way to do this...
    state[1:pop].=1;
    state[rand((1:pop),(1,nseeds))].=2; #seeds should be exposed cases

    notdoneyet=true
    while i<=(predays+postdays) #for each simulation, do the following

        #updating the infection states
        epistep!(state, net, p, q, r)  #this calls the function EpiSim.epistep! to update the state of all nodes

        #count the respective totals
        st[i,j] = count(x->x==1, state)
        ex[i,j] = count(x->x==2, state)
        fe[i,j] = count(x->x==3, state)
        rm[i,j] = count(x->x==4, state)
        
    
        inftot = pop-(st[i,j]+ex[i,j])   #if some condition is met, switch parameters, realign everything and continue
        if notdoneyet && inftot>feth   #notdoneyet - we only want to do this ONCE
            net=net2 #change network structure
            r=r2     #change the removal rate
            p=p2     #and the infection rate
            #realign everything - this is clunky, should be a nicer way to achieve the same thing...
            epirealign!(st, ex, fe, rm, i, j, predays, pop)
            i=predays           #counter set to done predays.
            notdoneyet=false    #we don't want to come back here again
        end
        
        i =+1
        
    end

end
    